In [233]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression

X, y = make_regression(n_samples=1000, n_features=14, n_informative=10, noise=15, random_state=42)
X = pd.DataFrame(X)
y = pd.Series(y)
X.columns = [f'col_{col}' for col in X.columns]

In [237]:
class MyLineReg():
    def __init__(self, n_iter, learning_rate, metric=None, reg=None, l1_coef=None, l2_coef=None):
        """ 
        metric: r2, mse, mae, mape, rmse
        """
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.weights = None
        self.metric = metric
        self.reg = reg
        self.l1_coef = l1_coef
        self.l2_coef = l2_coef

    def predict(self, X):
        X = X.copy()
        X.insert(0, "bias", 1)
        return X @ self.weights
    
    def get_loss_metric(self, y_pred, y):
        error = y_pred - y
        if (self.metric == None) | (self.metric == 'mse'):
            return np.sum(error ** 2) / len(y_pred)
        elif self.metric == 'rmse':
            return np.sqrt(np.sum(error ** 2) / len(y_pred))
        elif self.metric == 'mae':
            return np.sum(abs(error)) / len(y_pred)
        elif self.metric == 'mape':
            return 100 / len(y_pred) * np.sum(abs((error) / y))
        elif self.metric == 'r2':
            return 1 - np.sum((error) ** 2) / np.sum((y - np.mean(y)) ** 2)
        raise ValueError('Invalid metric. Supported metrics: mae, mse, rmse, mape, r2')
    
    def get_loss_regularization(self):
        if self.reg is None:
            return 0
        elif self.reg == 'l1':
            return self.l1_coef * np.sum(abs(self.weights))
        elif self.reg == 'l2':
            return self.l2_coef * np.sum(self.weights ** 2)
        elif self.reg == 'elasticnet':
            return self.l1_coef * np.sum(abs(self.weights)) + self.l2_coef * np.sum(self.weights ** 2)
        raise ValueError('Invalid regularization. Supported methods: l1, l2, elasticnet')

    def gr_mse(self, X, y_pred, y):
        error = y_pred - y
        if self.reg is None:
            return (2 / len(y_pred)) * X.T @ error
        if self.reg == 'l1':
            return (2 / len(y_pred)) * X.T @ error + self.l1_coef * (self.weights / abs(self.weights))
        elif self.reg == 'l2':
            return (2 / len(y_pred)) * X.T @ error + self.l2_coef * 2 * self.weights
        elif self.reg == 'elasticnet':
            return (2 / len(y_pred)) * X.T @ error + self.l1_coef * (self.weights / abs(self.weights)) + self.l2_coef * 2 * self.weights
        raise ValueError('Invalid regularization. Supported methods: l1, l2, elasticnet')

    def fit(self, X, y, verbose=False):
        X = X.copy()
        y = y.copy()
        X.insert(0, "bias", 1)
        if self.weights is None:    
            self.weights = np.ones(X.shape[1])

        for epoch in range(1, self.n_iter + 1):
            step = self.learning_rate if type(self.learning_rate) in (int, float) else self.learning_rate(epoch)
            y_pred = X @ self.weights
            self.weights -= step * self.gr_mse(X, y_pred, y)

            self.loss_func_score = self.get_loss_metric(y_pred, y) + self.get_loss_regularization()
            if verbose and ((epoch % verbose == 0) | (epoch == 1)):
                if self.metric:
                    print(f"{epoch} | loss: {self.loss_func_score} | lr: {np.round(step, 5)}")
                else:
                    print(f"{epoch} | mse loss: {self.loss_func_score} | lr: {np.round(step, 5)}")
        self.best_score = self.loss_func_score

    def get_coef(self):
        return self.weights[1:]
    
    def get_best_score(self):
        return self.best_score

    def __str__(self) -> str:
        return f"{self.__class__.__name__} class: " + ", ".join("%s=%s" % item for item in vars(self).items())
    
    def __repr__(self) -> str:
        return f"{self.__class__.__name__} class: " + ", ".join("%s=%s" % item for item in vars(self).items())

In [241]:
reg = MyLineReg(100, metric='mae', learning_rate=lambda iter: 0.5 * (0.85 ** iter))

In [242]:
reg.fit(X, y, 10)

1 | loss: 113.7842420167003 | lr: 0.425
10 | loss: 12.01282802500864 | lr: 0.09844
20 | loss: 11.99871451957513 | lr: 0.01938
30 | loss: 11.997663526375304 | lr: 0.00382
40 | loss: 11.997483371595672 | lr: 0.00075
50 | loss: 11.997450131422381 | lr: 0.00015
60 | loss: 11.997443614110333 | lr: 3e-05
70 | loss: 11.99744233205506 | lr: 1e-05
80 | loss: 11.997442079691346 | lr: 0.0
90 | loss: 11.997442030008948 | lr: 0.0
100 | loss: 11.997442020227814 | lr: 0.0


In [243]:
reg.get_best_score()

11.997442020227814